In [2]:
import cv2
import numpy as np
import os
import mediapipe as mp
import json

# 設定 (変更なし)
DATA_PATH = os.path.join(os.getcwd(), 'MP_Data_JSON')
actions = np.array(['heel_hook', 'deadpoint', 'dyno','cross_move'])
no_videos = 8
sequence_length = 180

# Mediapipe Holisticモデルの初期化 (変更なし)
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils


# 取得したい姿勢ランドマークのインデックスを指定
DESIRED_POSE_LANDMARKS = [12, 14, 16, 18, 20, 22] # ユーザー指定のものを維持
# 取得したい顔ランドマークのインデックスを指定 (空リストのままなら顔は処理されません)
DESIRED_FACE_LANDMARKS = []
# 取得したい左手ランドマークのインデックスを指定 (空リストのままなら左手は処理されません)
DESIRED_LEFT_HAND_LANDMARKS = []
# 取得したい右手ランドマークのインデックスを指定 (空リストのままなら右手は処理されません)
DESIRED_RIGHT_HAND_LANDMARKS = []

def multiple_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    image_height, image_width, _ = image.shape

    # 姿勢のキーポイントの描画
    if results.pose_landmarks:
        # 点の描画
        for idx in DESIRED_POSE_LANDMARKS: # グローバル変数を参照
            if idx < len(results.pose_landmarks.landmark): # インデックス範囲チェック
                landmark = results.pose_landmarks.landmark[idx]
                # visibility が低いものは描画しないなどの条件も追加可能
                # if landmark.visibility < 0.01: # 例: visibilityが極端に低いものはスキップ
                #     continue
                cx = int(landmark.x * image_width)
                cy = int(landmark.y * image_height)
                # ランドマーク（点）のスタイル: 青色、半径を調整
                cv2.circle(image, (cx, cy), 3, (0, 0, 255), cv2.FILLED)

        # 線の描画 (POSE_CONNECTIONS を元に、指定ランドマーク間のみ)
        if mp_holistic.POSE_CONNECTIONS: # POSE_CONNECTIONS が定義されている場合
            for connection in mp_holistic.POSE_CONNECTIONS:
                start_idx = connection[0]
                end_idx = connection[1]

                # 接続する両方の点が DESIRED_POSE_LANDMARKS に含まれている場合のみ描画
                if start_idx in DESIRED_POSE_LANDMARKS and end_idx in DESIRED_POSE_LANDMARKS:
                    # ランドマークの存在確認
                    if start_idx < len(results.pose_landmarks.landmark) and \
                       end_idx < len(results.pose_landmarks.landmark):

                        start_landmark = results.pose_landmarks.landmark[start_idx]
                        end_landmark = results.pose_landmarks.landmark[end_idx]

                        # visibility が低いものは描画しないなどの条件も追加可能
                        # if start_landmark.visibility < 0.01 or end_landmark.visibility < 0.01:
                        #     continue

                        start_point = (int(start_landmark.x * image_width), int(start_landmark.y * image_height))
                        end_point = (int(end_landmark.x * image_width), int(end_landmark.y * image_height))
                        # コネクション（線）のスタイル: 緑色、太さを調整
                        cv2.line(image, start_point, end_point, (51, 255, 51), 2)

    # 顔、左手、右手の描画は DESIRED_..._LANDMARKS が空なので、ここでは何もしません。
    # もし特定部位を描画したい場合は、同様のロジックを DESIRED_FACE_LANDMARKS などを使って追加します。


# ★★★ 変更点: JSON形式でキーポイントを抽出 ★★★
def extract_keypoints(results):
    keypoints_data = {}

    # >>>>>>>>>> 変更点 3: 関数内の DESIRED_... リスト定義を削除 (グローバル変数を参照するため) <<<<<<<<<<<<
    # DESIRED_POSE_LANDMARKS = [12 , 14 , 16 , 18 , 20 , 22] # これらの行を削除
    # DESIRED_FACE_LANDMARKS = []
    # DESIRED_LEFT_HAND_LANDMARKS = []
    # DESIRED_RIGHT_HAND_LANDMARKS = []
    # >>>>>>>>>> 変更点 3: ここまで <<<<<<<<<<<<

    if results.pose_landmarks:
        pose_landmarks = []
        for i, res in enumerate(results.pose_landmarks.landmark):
            if i in DESIRED_POSE_LANDMARKS: # グローバル変数を参照
                pose_landmarks.append({
                    "id": i,
                    "x": res.x,
                    "y": res.y,
                    "z": res.z,
                    "visibility": res.visibility
                })
        keypoints_data["pose"] = pose_landmarks
    else:
        keypoints_data["pose"] = []

    if results.face_landmarks:
        face_landmarks = []
        if DESIRED_FACE_LANDMARKS: # グローバル変数を参照
            for i, res in enumerate(results.face_landmarks.landmark):
                if i in DESIRED_FACE_LANDMARKS:
                    face_landmarks.append({
                        "id": i, "x": res.x, "y": res.y, "z": res.z
                    })
        keypoints_data["face"] = face_landmarks # DESIRED_FACE_LANDMARKS が空なら空リストが入る
    else:
        keypoints_data["face"] = []

    if results.left_hand_landmarks:
        lh_landmarks = []
        if DESIRED_LEFT_HAND_LANDMARKS: # グローバル変数を参照
            for i, res in enumerate(results.left_hand_landmarks.landmark):
                if i in DESIRED_LEFT_HAND_LANDMARKS:
                    lh_landmarks.append({
                        "id": i, "x": res.x, "y": res.y, "z": res.z
                    })
        keypoints_data["left_hand"] = lh_landmarks
    else:
        keypoints_data["left_hand"] = []

    if results.right_hand_landmarks:
        rh_landmarks = []
        if DESIRED_RIGHT_HAND_LANDMARKS: # グローバル変数を参照
            for i, res in enumerate(results.right_hand_landmarks.landmark):
                if i in DESIRED_RIGHT_HAND_LANDMARKS:
                    rh_landmarks.append({
                        "id": i, "x": res.x, "y": res.y, "z": res.z
                    })
        keypoints_data["right_hand"] = rh_landmarks
    else:
        keypoints_data["right_hand"] = []

    return keypoints_data

# データの収集 (以降のコードは変更なし)
# OpenCVのウィンドウ表示設定 (変更なし)
cv2.namedWindow('OpenCV Feed', cv2.WINDOW_NORMAL)
cv2.resizeWindow('OpenCV Feed', (1280, 720))

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for action in actions:
        action_path = os.path.join(DATA_PATH, action)
        if not os.path.exists(action_path):
            os.makedirs(action_path)

        for video_num in range(1, no_videos + 1):
            video_file_path = os.path.join(r'C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos', action + str(video_num) + '.mp4')
            cap = cv2.VideoCapture(video_file_path)

            video_data_save_path = os.path.join(action_path, str(video_num))
            if not os.path.exists(video_data_save_path):
                os.makedirs(video_data_save_path)

            print(f'Processing video: {video_file_path}')
            frames_collected_for_current_video = 0

            for frame_num in range(sequence_length):
                ret, frame = cap.read()
                if not ret:
                    print(f"Warning: Video {video_file_path} ended before collecting {sequence_length} frames. Collected {frame_num} frames.")
                    break

                image, results = multiple_detection(frame, holistic)

                draw_styled_landmarks(image, results) # 変更された関数が呼ばれる

                cv2.putText(image, f'Collecting frames for {action} - Video {video_num} - Frame {frame_num + 1}/{sequence_length}',
                            (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)
                cv2.imshow('OpenCV Feed', image)

                keypoints = extract_keypoints(results) # 変更された関数が呼ばれる
                json_path = os.path.join(video_data_save_path, f'{frame_num}.json')
                with open(json_path, 'w') as f:
                    json.dump(keypoints, f, indent=4)

                frames_collected_for_current_video +=1

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

            cap.release()
            if frames_collected_for_current_video < sequence_length:
                 print(f"Warning: For video {video_file_path}, only {frames_collected_for_current_video}/{sequence_length} frames were saved to {video_data_save_path}.")

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook1.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook2.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook3.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook4.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook5.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook6.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos\heel_hook7.mp4
Processing video: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\videos